In [1]:
import acl
import numpy as np
import struct
from constant import ACL_MEM_MALLOC_NORMAL_ONLY, \
    ACL_MEMCPY_HOST_TO_DEVICE, ACL_MEMCPY_DEVICE_TO_HOST, \
    ACL_ERROR_NONE, NPY_BYTE
from PIL import Image
from postprocessing import estimate_paf, draw_humans
from acl_sample import Sample

In [3]:
sample = Sample(0,
                "./model/openpose_from_tf.om",
                368,
                368)

[Sample] init resource stage:
[Sample] init resource stage success
[Model] class Model init resource stage:
model output size 3
output  0
model output dims ({'name': 'peaks:0', 'dimCount': 4, 'dims': [1, 92, 92, 19]}, 0)
model output datatype 0
output  1
model output dims ({'name': 'heatmat_output:0', 'dimCount': 4, 'dims': [1, 92, 92, 19]}, 0)
model output datatype 0
output  2
model output dims ({'name': 'pafmat_output:0', 'dimCount': 4, 'dims': [1, 92, 92, 38]}, 0)
model output datatype 0
[Model] create model output dataset:
[Model] create model output dataset success
[Model] class Model init resource stage success


In [6]:
img_dict = {"path": "./data/p3.jpg", "dtype": np.uint8}

In [7]:
model_output = sample.forward(img_dict)

[Sample] width:600 height:471
[Sample] image:./data/p3.jpg
[Dvpp] vpc decode stage:
[Dvpp] vpc decode stage success
[Dvpp] vpc resize stage:
[Dvpp] vpc resize stage success
dvpp_output_size 203136
[Model] create model input dataset:
[Model] create model input dataset success
[Model] execute stage:
[Model] execute stage success


Exception ignored in: <bound method Sample.__del__ of <acl_sample.Sample object at 0xfffefc3a4358>>
Traceback (most recent call last):
  File "/home/HwHiAiUser/pyACL_samples/acl_execute_model/acl_dvpp_openpose/acl_sample.py", line 48, in __del__
    if self.model_process:
AttributeError: 'Sample' object has no attribute 'model_process'
Exception ignored in: <bound method Model.__del__ of <acl_model.Model object at 0xfffefc3a4a20>>
Traceback (most recent call last):
  File "/home/HwHiAiUser/pyACL_samples/acl_execute_model/acl_dvpp_openpose/acl_model.py", line 34, in __del__
    self._release_dataset()
  File "/home/HwHiAiUser/pyACL_samples/acl_execute_model/acl_dvpp_openpose/acl_model.py", line 113, in _release_dataset
    for dataset in [self.input_dataset, self.output_data]:
AttributeError: 'Model' object has no attribute 'input_dataset'


In [ ]:
num_output = acl.mdl.get_dataset_num_buffers(model_output)

def get_model_output_by_index(model_output, i):
    temp_output_buf = acl.mdl.get_dataset_buffer(model_output, i)

    infer_output_ptr = acl.get_data_buffer_addr(temp_output_buf)
    infer_output_size = acl.get_data_buffer_size(temp_output_buf)
#     print("infer_output_size", infer_output_size)
    
#     output_host, _ = acl.rt.malloc_host(infer_output_size)
#     acl.rt.memcpy(output_host, infer_output_size, infer_output_ptr,
#                           infer_output_size, ACL_MEMCPY_DEVICE_TO_HOST)
    
    result = acl.util.ptr_to_numpy(infer_output_ptr, (infer_output_size,), NPY_BYTE)
    return np.array(struct.unpack(f"{infer_output_size//4}f", bytearray(result)), dtype=np.float32).reshape(92, 92, -1)

In [ ]:
peaks = get_model_output_by_index(model_output, 0)
heatmat = get_model_output_by_index(model_output, 1)
pafmat = get_model_output_by_index(model_output, 2)

In [ ]:
humans = estimate_paf(peaks, heatmat, pafmat)

In [ ]:
img = draw_humans(np.array(Image.open(img_dict["path"])), humans)

In [ ]:
Image.fromarray(img)